## Import Necessary Libraries


In [ ]:
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore
from sklearn import preprocessing
import numpy as np
from keras import backend as K

## Data load and overview

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/Sumit-saha12/Shipment-price-prediction_project_iNeuron_project/main/SCMS_Delivery_History_Dataset.csv')
data.head()

,ID,Project Code,PQ #,PO / SO #,ASN/DN #,Country,Managed By,Fulfill Via,Vendor INCO Term,Shipment Mode,...,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,First Line Designation,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,...,30,19,551.0,29.00,0.97,Ranbaxy Fine Chemicals LTD,Yes,13,780.34,NaN
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,...,240,1000,6200.0,6.20,0.03,"Aurobindo Unit III, India",Yes,358,4521.5,NaN
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,...,100,500,40000.0,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,Yes,171,1653.78,NaN
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,31920,127360.8,3.99,0.07,"Ranbaxy, Paonta Shahib, India",Yes,1855,16007.06,NaN
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,38000,121600.0,3.20,0.05,"Aurobindo Unit III, India",Yes,7590,45450.08,NaN


## See Data types

In [ ]:
data.dtypes

ID                                int64
Project Code                     object
PQ #                             object
PO / SO #                        object
ASN/DN #                         object
Country                          object
Managed By                       object
Fulfill Via                      object
Vendor INCO Term                 object
Shipment Mode                    object
PQ First Sent to Client Date     object
PO Sent to Vendor Date           object
Scheduled Delivery Date          object
Delivered to Client Date         object
Delivery Recorded Date           object
Product Group                    object
Sub Classification               object
Vendor                           object
Item Description                 object
Molecule/Test Type               object
Brand                            object
Dosage                           object
Dosage Form                      object
Unit of Measure (Per Pack)        int64
Line Item Quantity                int64


In [ ]:
data.shape

(10324, 33)

In [ ]:
data.describe()

,ID,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Line Item Insurance (USD)
count,10324.000000,10324.000000,10324.000000,1.032400e+04,10324.000000,10324.000000,10037.000000
mean,51098.968229,77.990895,18332.534870,1.576506e+05,21.910241,0.611701,240.117626
std,31944.332496,76.579764,40035.302961,3.452921e+05,45.609223,3.275808,500.190568
min,1.000000,1.000000,1.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,12795.750000,30.000000,408.000000,4.314593e+03,4.120000,0.080000,6.510000
50%,57540.500000,60.000000,3000.000000,3.047147e+04,9.300000,0.160000,47.040000
75%,83648.250000,90.000000,17039.750000,1.664471e+05,23.592500,0.470000,252.400000
max,86823.000000,1000.000000,619999.000000,5.951990e+06,1345.640000,238.650000,7708.440000


## Unnecessary Atrributes Remove

In [ ]:
new_data=data.drop([
    'ID',
    'Project Code',
    'PQ #',
    'PO / SO #',
    'ASN/DN #',
    'Country',
    'Managed By',
    'Fulfill Via',
    'Vendor INCO Term',
    'Shipment Mode',
    'PQ First Sent to Client Date',
    'PO Sent to Vendor Date',
    'Scheduled Delivery Date',
    'Delivered to Client Date',
    'Delivery Recorded Date',
    'Vendor',
    'Item Description',
    'Manufacturing Site',
    'Product Group',
    'Sub Classification',
    'Molecule/Test Type',
    'Brand',
    'Dosage',
    'Dosage Form'
    ],axis=1)

In [ ]:
new_data.shape

(10324, 9)

In [ ]:
new_data.describe()

,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Line Item Insurance (USD)
count,10324.000000,10324.000000,1.032400e+04,10324.000000,10324.000000,10037.000000
mean,77.990895,18332.534870,1.576506e+05,21.910241,0.611701,240.117626
std,76.579764,40035.302961,3.452921e+05,45.609223,3.275808,500.190568
min,1.000000,1.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,30.000000,408.000000,4.314593e+03,4.120000,0.080000,6.510000
50%,60.000000,3000.000000,3.047147e+04,9.300000,0.160000,47.040000
75%,90.000000,17039.750000,1.664471e+05,23.592500,0.470000,252.400000
max,1000.000000,619999.000000,5.951990e+06,1345.640000,238.650000,7708.440000


In [ ]:
new_data['Freight Cost (USD)'].unique()

array(['780.34', '4521.5', '1653.78', ..., '26180', '3410',
       'See DN-4282 (ID#:83919)'], dtype=object)

In [ ]:
new_data=new_data[new_data['First Line Designation']!='No']
# new_data=new_data[new_data['Line Item Insurance (USD)']!=0]
new_data=new_data[new_data['Freight Cost (USD)']!='Freight Included in Commodity Cost']
new_data=new_data[new_data['Freight Cost (USD)']!='Invoiced Separately']
new_data=new_data[new_data['Weight (Kilograms)']!='Weight Captured Separately']
new_data=new_data.dropna()
# new_data

In [ ]:
new_data=new_data.drop(['First Line Designation'],axis=1)

In [ ]:
new_data['Freight Cost (USD)']=pd.to_numeric(new_data['Freight Cost (USD)'])
new_data['Weight (Kilograms)']=pd.to_numeric(new_data['Weight (Kilograms)'])
new_data.dtypes

Unit of Measure (Per Pack)      int64
Line Item Quantity              int64
Line Item Value               float64
Pack Price                    float64
Unit Price                    float64
Weight (Kilograms)              int64
Freight Cost (USD)            float64
Line Item Insurance (USD)     float64
dtype: object

In [ ]:
# new_data=pd.get_dummies(new_data)

In [ ]:
new_data.describe()

,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
count,5992.000000,5992.000000,5.992000e+03,5992.000000,5992.000000,5992.000000,5992.000000,5992.00000
mean,76.333611,21917.236482,1.953564e+05,23.928667,0.756891,3273.743825,11272.019543,296.92088
std,79.540002,43486.860613,3.773931e+05,46.380587,4.110614,7039.340764,15965.589608,553.33782
min,1.000000,1.000000,0.000000e+00,0.000000,0.000000,0.000000,0.750000,0.00000
25%,30.000000,1000.000000,1.041875e+04,4.300000,0.090000,216.750000,2181.512500,15.83000
50%,60.000000,4927.000000,5.982163e+04,9.500000,0.160000,1093.500000,6011.310000,90.53500
75%,100.000000,23125.000000,2.267010e+05,27.000000,0.660000,3416.000000,14547.550000,336.57000
max,1000.000000,619999.000000,5.951990e+06,1250.000000,238.650000,154780.000000,289653.200000,7708.44000


## Min-Max Scaling for reduce outliers

In [ ]:
# new_data=new_data.apply(zscore)




for i in new_data:
  new_data[i]=((new_data[i]-min(new_data[i]))/(max(new_data[i])-min(new_data[i])))




  # new_data[i]=(new_data[i])/(max(new_data[i]))
  # new_data[i]=np.array(new_data[i])

# min, max = min(new_data), max(new_data)

# new_data = new_data.apply(lambda x: (x - min)/(max - min))

In [ ]:
new_data.describe()

,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
count,5992.000000,5992.000000,5992.000000,5992.000000,5992.000000,5992.000000,5992.000000,5992.000000
mean,0.075409,0.035349,0.032822,0.019143,0.003172,0.021151,0.038913,0.038519
std,0.079620,0.070140,0.063406,0.037104,0.017224,0.045480,0.055120,0.071783
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.029029,0.001611,0.001750,0.003440,0.000377,0.001400,0.007529,0.002054
50%,0.059059,0.007945,0.010051,0.007600,0.000670,0.007065,0.020751,0.011745
75%,0.099099,0.037297,0.038088,0.021600,0.002766,0.022070,0.050222,0.043663
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
x = new_data.drop(['Pack Price'],axis=1)
y1= new_data['Pack Price']
# y2= new_data['Unit Price']

In [ ]:
a=[x,y1]
for i in a:
  print(i.shape)

(5992, 7)
(5992,)


In [ ]:
x_train,x_test,y1_train,y1_test = train_test_split(x,y1,test_size=0.2,random_state=0)
# x_train,x_test,y2_train,y2_test = train_test_split(x,y2,test_size=0.2,random_state=0)

In [ ]:
b=[x_train,x_test,y1_train,y1_test]
c=['x_train','x_test','y1_train','y1_test']
for i in range(len(b)):
  print('shape of {} is {}'.format(c[i],b[i].shape))

shape of x_train is (4793, 7)
shape of x_test is (1199, 7)
shape of y1_train is (4793,)
shape of y1_test is (1199,)


In [ ]:
max(new_data['Line Item Value'])

1.0

In [ ]:
# for i in x_train:
#   x_train[i]=(x_train[i]-min(x_train[i]))/(max(x_train[i])-min(x_train[i]))

# y1_train=(y1_train-min(y1_train))/(max(y1_train)-min(y1_train))
# y2_train=(y2_train-min(y2_train))/(max(y2_train)-min(y2_train))
# for i in x_train:
#   x_train[i]=(x_train[i])/(max(x_train[i]))

# y1_train=(y1_train)/(max(y1_train))
# y2_train=(y2_train)/(max(y2_train))

In [ ]:
x_train.describe()

,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Unit Price,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
count,4793.000000,4793.000000,4793.000000,4793.000000,4793.000000,4793.000000,4793.000000
mean,0.076539,0.035419,0.032682,0.003261,0.020992,0.038129,0.038400
std,0.081819,0.070464,0.063398,0.018587,0.044850,0.052230,0.072586
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000047,0.000000
25%,0.029029,0.001611,0.001735,0.000377,0.001376,0.007451,0.002034
50%,0.059059,0.007856,0.009995,0.000670,0.006978,0.020747,0.011533
75%,0.099099,0.037500,0.037766,0.002598,0.022193,0.050020,0.043389
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
def r2_score(y_test, y_pred):
    SS_res =  K.sum(K.square( y_test-y_pred )) 
    SS_tot = K.sum(K.square( y_test - K.mean(y_test) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )


In [ ]:
class callback(keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if(logs.get('r2_score')>0.98):
      print("\nAccuracy over 98% that's why training is stop")
      self.model.stop_training=True


## Model Training

In [ ]:
lr = keras.layers.LeakyReLU()
call=callback()
  
model = keras.Sequential([
    keras.layers.Dense(150,input_shape=[7],activation='relu'),
    keras.layers.Dense(50,activation='relu'),
    keras.layers.Dense(20,activation='relu'),
    keras.layers.Dense(1,activation='linear')
  ])

model.compile(optimizer='adam',
                loss='mean_squared_error',
                metrics=['mae','mse',r2_score],
                )

model.fit(x_train,y1_train,epochs=100,callbacks=[call])
  # return model

Epoch 1/100
150/150 [==============================] - 9s 4ms/step - loss: 0.0011 - mae: 0.0158 - mse: 0.0011 - r2_score: 0.0054
Epoch 2/100
150/150 [==============================] - 1s 4ms/step - loss: 8.6284e-04 - mae: 0.0113 - mse: 8.6284e-04 - r2_score: 0.4585
Epoch 3/100
150/150 [==============================] - 1s 4ms/step - loss: 7.9180e-04 - mae: 0.0106 - mse: 7.9180e-04 - r2_score: 0.4675
Epoch 4/100
150/150 [==============================] - 1s 3ms/step - loss: 7.1253e-04 - mae: 0.0097 - mse: 7.1253e-04 - r2_score: 0.5484
Epoch 5/100
150/150 [==============================] - 1s 4ms/step - loss: 5.7004e-04 - mae: 0.0079 - mse: 5.7004e-04 - r2_score: 0.6704
Epoch 6/100
150/150 [==============================] - 1s 5ms/step - loss: 5.5184e-04 - mae: 0.0088 - mse: 5.5184e-04 - r2_score: 0.5932
Epoch 7/100
150/150 [==============================] - 1s 5ms/step - loss: 4.7958e-04 - mae: 0.0085 - mse: 4.7958e-04 - r2_score: 0.5191
Epoch 8/100
150/150 [============================

In [ ]:
model.evaluate(x_test,y1_test)

38/38 [==============================] - 0s 4ms/step - loss: 2.6346e-05 - mae: 0.0039 - mse: 2.6346e-05 - r2_score: 0.9465


[2.634569682413712e-05,
 0.003863639198243618,
 2.634569682413712e-05,
 0.9464871287345886]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 150)               1200      
                                                                 
 dense_1 (Dense)             (None, 50)                7550      
                                                                 
 dense_2 (Dense)             (None, 20)                1020      
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
Total params: 9,791
Trainable params: 9,791
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.save('new_model.h5')